In [40]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from string import punctuation
from collections import Counter
%matplotlib inline

In [41]:
from pandas import read_excel


In [27]:
data.head

<bound method NDFrame.head of                                                   Message  Code
771070                                      Yebo Daniel       0
414181  Please read through this and let your tutor kn...     0
735824  Which letters create the future tense in Zulua...     0
481008                                Sanibona ThishaBot      1
438886                                              Ready     1
31516                                       forgot sorry      1
688243  Watch this video if you need help external You...     0
362810                                               Now      1
438554                                              Ready     1
19619                                           Sawubona      0
690767                            We can do double lesson     1
390871                                         Okay cool      1
234402                                 It was my pleasure     1
349263                        no problem ngiyaphila nami      1
445660    

In [47]:
data = read_excel('real_data3.xlsx')

In [48]:
data.head()

,Message,Code
0,I have classes,1
1,it was nice seeing u too,1
2,hectic right,1
3,lol,1
4,too much studying,1


In [49]:
from sklearn.utils import shuffle
data = shuffle(data)

In [50]:
labels = data.iloc[:, 1].values
reviews = data.iloc[:, 0].values

In [51]:
reviews_processed = [] 
for review in reviews:
    review_cool_one = ''.join([char for char in review if char not in punctuation])
    reviews_processed.append(review_cool_one)

In [52]:
word_reviews = []
all_words = []
for review in reviews_processed:
    word_reviews.append(review.lower().split())
    for word in review.split():
        all_words.append(word.lower())
   
counter = Counter(all_words)
vocab = sorted(counter, key=counter.get, reverse=True)

In [53]:
vocab_to_int = {word: i for i, word in enumerate(vocab, 1)}

In [54]:
reviews_to_ints = []
for review in word_reviews:
    reviews_to_ints.append([vocab_to_int[word] for word in review])

In [55]:
reviews_lens = Counter([len(x) for x in reviews_to_ints])
print('Zero-length {}'.format(reviews_lens[0]))
print("Max review length {}".format(max(reviews_lens)))

Zero-length 0
Max review length 348


In [56]:
seq_len = 250

features = np.zeros((len(reviews_to_ints), seq_len), dtype=int)
for i, review in enumerate(reviews_to_ints):
    features[i, -len(review):] = np.array(review)[:seq_len]

In [57]:
X_train = features[:840522]
y_train = labels[:840522]

X_test = features[840522:]
y_test = labels[84522:]


print('X_trian shape {}'.format(X_train.shape))


X_trian shape (840522, 250)


In [58]:
hidden_layer_size = 512 # how many nodes LSTM cells will have
number_of_layers = 1 # how many RNN layers the network will use
batch_size = 100 # how many reviews we feed at onces
learning_rate = 0.001 # learning rate
number_of_words = len(vocab_to_int) + 1 #how many unique words do we have in vocab (+1  is used for 0 - padding)
dropout_rate = 0.8 
embed_size = 300 #how long our word embedings will be
epochs = 6 # how many epochs do we use for training

In [59]:
tf.reset_default_graph() #Clean the graph

In [60]:
inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
targets = tf.placeholder(tf.int32, [None, None], name='targets')

In [61]:
word_embedings = tf.Variable(tf.random_uniform((number_of_words, embed_size), -1, 1))
embed = tf.nn.embedding_lookup(word_embedings, inputs)

In [62]:
hidden_layer = tf.contrib.rnn.BasicLSTMCell(hidden_layer_size)
hidden_layer = tf.contrib.rnn.DropoutWrapper(hidden_layer, dropout_rate)

cell = tf.contrib.rnn.MultiRNNCell([hidden_layer]*number_of_layers)
init_state = cell.zero_state(batch_size, tf.float32)

In [63]:
outputs, states = tf.nn.dynamic_rnn(cell, embed, initial_state=init_state)

In [64]:
prediction = tf.layers.dense(outputs[:, -1], 1, activation=tf.sigmoid)
cost = tf.losses.mean_squared_error(targets, prediction)

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [65]:
currect_pred = tf.equal(tf.cast(tf.round(prediction), tf.int32), targets)
accuracy = tf.reduce_mean(tf.cast(currect_pred, tf.float32))

In [66]:
session = tf.Session()

In [67]:
session.run(tf.global_variables_initializer())

In [ ]:
for i in range(epochs):
    training_accurcy = []
    ii = 0
    epoch_loss = []
    while ii + batch_size <= len(X_train):
        X_batch = X_train[ii:ii+batch_size]
        y_batch = y_train[ii:ii+batch_size].reshape(-1, 1)
        
        a, o, _ = session.run([accuracy, cost, optimizer], feed_dict={inputs:X_batch, targets:y_batch})

        training_accurcy.append(a)
        epoch_loss.append(o)
        ii += batch_size
    print('Epoch: {}/{}'.format(i, epochs), ' | Current loss: {}'.format(np.mean(epoch_loss)),
          ' | Training accuracy: {:.4f}'.format(np.mean(training_accurcy)*100))

In [ ]:
test_accuracy = []

ii = 0
while ii + batch_size <= len(X_test):
    X_batch = X_test[ii:ii+batch_size]
    y_batch = y_test[ii:ii+batch_size].reshape(-1, 1)

    a = session.run([accuracy], feed_dict={inputs:X_batch, targets:y_batch})
    
    test_accuracy.append(a)
    ii += batch_size